今までの知見を使って、大量のURLを高速に取得するノートブック  
1ページにおけるURL取得量が増えれば増えるほど指数関数的に時間がかかるので、  
一ページで5000件取得するとかはやはり難しいといえる。  

- パーサーにlxmlを使う
- while文にlen(url)を使う
- browser.excute_scriptと、soup=でURLの取得は増やせる
- DBに格納するのは早めに行う
- while文前後の差集合で取得する？

In [ ]:
%reload_ext autoreload
%autoreload 2
import sys; sys.path.append('./src')
import urllib.request as urllib
from bs4 import BeautifulSoup
import re
from contextlib import closing
import sqlite3
import time
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from DB import *

In [ ]:
options = Options()
options.add_argument('--headless') 
browser = webdriver.Chrome(chrome_options=options)
DB_HEAD='../data/database/'

road_dbname=f'{DB_HEAD}1105_review_of_bakawall7.db'
low_dbname=f'{DB_HEAD}1122_low_rate_bakawall_nolimit_prodcuts.db'
high_dbname=f'{DB_HEAD}1122_high_rate_bakawall_nolimit_prodcuts.db'
Hconn,Hc=get_db_conn(high_dbname)
Lconn,Lc=get_db_conn(low_dbname)
head='https://www.amazon.co.jp'
# create_table = '''create table products (id,name,rate,url,review,text,pn)'''
# Hc.execute(create_table)
# Lc.execute(create_table)

def main(start_num=0):
    Rconn,Rc=get_db_conn(road_dbname)
    low_select_Rdb=f'''select date,rate,url from reviews where rate<3'''
    high_select_Rdb=f'''select date,rate,url from reviews where rate>3'''
    get_one_people_each_products_url(Rconn,Rc,Hconn,Hc,high_select_Rdb,start_num)
    get_one_people_each_products_url(Rconn,Rc,Lconn,Lc,low_select_Rdb,start_num)

In [ ]:

def scroll_and_geturl(browser,max_url=500):
    lenurl=0
    while lenurl<=max_url:
        browser.execute_script('scroll(0, document.body.scrollHeight)')
        soup = BeautifulSoup(browser.page_source, "lxml")
        review_box=soup.select('.profile-at-card')
        review_url=['https://www.amazon.co.jp'+x.select('.profile-at-review-link')[0].get('href') for x in review_box]
        lenurl=len(review_url)
        print(len(review_url),end='|')


In [ ]:
browser = webdriver.Chrome(chrome_options=options)
def get_one_people_each_products_url(Rconn,Rc,Wconn,Wc,select_sql,start_num=0):
    '''
    レビュワーURLを一気に受け取り、一つずつfor文を回す。
    その後一度にレビューURLを取得し、新しいDBに格納する
    各行にそのレビュアーの名前・商品名（NEW)・バカの壁につけた星・商品URLを記録する
    Parameters
    ------------------
    Rconn,Rc:読み込むDBのコントローラー。レビュワーURLが格納されているDB
    Wconn,Wc:書き出すDBのコントローラー。プロダクトURLが格納される予定のDB
    select_sql:読み込むDBにどんな指示をだすか。例えば星4以上のレビュアーのみに絞るなど
    '''
    
    for i,row in enumerate(Rc.execute(select_sql)):
        if i<=start_num:
            continue
        
        print(f'{i}人目、{row[0]}さんのレビューを取得中です')
        
        browser.get('https://www.amazon.co.jp'+row[2])
        product_url,product_name=get_product_url(browser,max_url=500)
        id_=[x for x in range(1,len(product_url)+1)]
        datas=[(n,i,r,pu,pn) for n,i,r,pu,pn in zip([row[0]]*len(product_url),id_,[row[1]]*len(product_url),product_url,product_name)]

        insert_sql = 'insert into products (name,id,rate,url,pn) values (?,?,?,?,?)'
        #↑順番が違う
        Wc.executemany(insert_sql, datas)
        Wconn.commit()

def get_product_url(browser,max_url):
    '''
    具体的にはURLを取得してHTMLを取得する部分
    '''
    '''-------------------------------------------------------------------'''
    lenurl=0
    break_power=0

    def igonre_none_product(x):
        try:
            return x.select('.profile-at-product-title-container > span')[0].text
        except:
            return('no_product')
    '''-------------------------------------------------------------------'''
    soup = BeautifulSoup(browser.page_source, "lxml")
    try:
        review_number=soup.select('.dashboard-desktop-stat-value')[1].text
    except:
        review_number=999999
    print(f'レビューの数は{review_number}。')
    
    
    
    while lenurl<=max_url:
        browser.execute_script('scroll(0, document.body.scrollHeight)')
        soup = BeautifulSoup(browser.page_source, "lxml")
        review_box=soup.select('.profile-at-card')
        review_url=['https://www.amazon.co.jp'+x.select('.profile-at-review-link')[0].get('href') for x in review_box]
        products_name=[igonre_none_product(x) for x in review_box]    
        lenurl2=len(review_url)
        
        if lenurl+10==lenurl2:
            break_power=0
            
        if lenurl==lenurl2:
            break_power+=1
        
        lenurl=lenurl2
        
        if break_power>=5:
            print('Break!')
            return(review_url,products_name)

        print(len(review_url),end='|')
    
    return(review_url,products_name)


In [ ]:
main()